#  Principal Component Analysis

Andiamo ora ad approfondire quello che è forse uno degli algoritmi non supervisionati più utilizzati, l'analisi delle componenti principali (PCA). 
PCA come abbiamo già visto è fondamentalmente un algoritmo di riduzione della dimensionalità, ma può anche essere utile come strumento per la visualizzazione, per il filtraggio del rumore, per l'estrazione e l'ingegneria delle funzionalità e molto altro ancora. 

## Entriamo nel dettaglio

L'analisi delle componenti principali come sappiamo è un metodo veloce e flessibile non supervisionato per la riduzione della dimensionalità nei dati. Il suo comportamento è più semplice da visualizzare osservando un set di dati bidimensionale, consideriamo quindi i seguenti 200 punti:

In [ ]:
#import library
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

#create point
rng = np.random.RandomState(1)
X = np.dot(rng.rand(2, 2), rng.randn(2, 200)).T
plt.scatter(X[:, 0], X[:, 1])
plt.axis('equal')

A prima vista, è chiaro che esiste una relazione quasi lineare tra le variabili x e y, ciò ricorda i dati di regressione lineare che abbiamo esplorato in precedenza, ma la impostazione del problema qui è leggermente diversa: invece di tentare di prevedere i valori y dai valori x, come sappiamo il problema di apprendimento non supervisionato tenta di apprendere la relazione tra i valori x e y.

Nell'analisi delle componenti principali, questa relazione viene quantificata trovando un elenco degli assi principali nei dati e utilizzando tali assi per descrivere il set di dati. Utilizzando lo stimatore PCA di Scikit-Learn, possiamo calcolarlo come segue:

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(X)

L'adattamento apprende alcune quantità dai dati, soprattutto i "componenti" e la "varianza spiegata":

In [ ]:
print(pca.components_)

In [ ]:
print(pca.explained_variance_)

Per vedere cosa significano questi numeri, visualizziamoli come vettori sui dati di input, utilizzando i "componenti" per definire la direzione del vettore e la "varianza spiegata" per definire la lunghezza quadrata del vettore:

In [ ]:
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->', linewidth=2,
                    shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

# plot data
plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v)
plt.axis('equal')

Questi vettori rappresentano gli assi principali dei dati e la lunghezza del vettore è un'indicazione di quanto "importante" sia quell'asse nel descrivere la distribuzione dei dati; più precisamente, è una misura della varianza dei dati quando proiettati su quell'asse. La proiezione di ciascun punto dati sugli assi principali sono le "componenti principali" dei dati.

Se tracciamo queste componenti principali accanto ai dati originali, vediamo i grafici mostrati qui:

![](dati/img/05.09-PCA-rotation.png)


Questa trasformazione dagli assi dei dati agli assi principali è una trasformazione affine , il che significa sostanzialmente che è composta da traslazione, rotazione e ridimensionamento uniforme.

Sebbene questo algoritmo per trovare i componenti principali possa sembrare solo una curiosità matematica, risulta avere applicazioni di vasta portata nel mondo dell'apprendimento automatico e dell'esplorazione dei dati.

### PCA e riduzione della dimensionalità

L'utilizzo della PCA per la riduzione della dimensionalità comporta l'azzeramento di uno o più dei componenti principali più piccoli, ottenendo una proiezione dei dati a dimensione inferiore che preserva la massima varianza dei dati.

Ecco un esempio di utilizzo della PCA come trasformazione di riduzione della dimensionalità:

In [ ]:
pca = PCA(n_components=1)
pca.fit(X)
X_pca = pca.transform(X)
print("original shape:   ", X.shape)
print("transformed shape:", X_pca.shape)

I dati trasformati sono stati ridotti a un'unica dimensione. Per comprendere l'effetto di questa riduzione della dimensionalità, possiamo eseguire la trasformazione inversa di questi dati ridotti e tracciarli insieme ai dati originali:

In [ ]:
X_new = pca.inverse_transform(X_pca)
plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
plt.scatter(X_new[:, 0], X_new[:, 1], alpha=0.8)
plt.axis('equal')

I punti azzurri sono i dati originali, mentre i punti arancioni sono la versione proiettata. Ciò rende chiaro cosa significa una riduzione della dimensionalità della PCA: le informazioni lungo l'asse o gli assi principali meno importanti vengono rimosse, lasciando solo i componenti dei dati con la varianza più elevata. La frazione di varianza che viene eliminata (proporzionale alla diffusione dei punti attorno alla linea formata in questa figura) è approssimativamente una misura di quanta "informazione" viene scartata in questa riduzione della dimensionalità.

Questo set di dati di dimensione ridotta è in un certo senso "abbastanza buono" per codificare le relazioni più importanti tra i punti: nonostante la riduzione della dimensione dei dati del 50%, la relazione complessiva tra i punti dati viene per lo più preservata.

### PCA per la visualizzazione: cifre scritte a mano

L'utilità della riduzione della dimensionalità potrebbe non essere del tutto evidente solo in due dimensioni, ma diventa molto più chiara quando si esaminano dati ad alta dimensionalità. Per capirlo, diamo una rapida occhiata pratica all'applicazione della PCA ai dati in cifre scritte a mano di cui abbiamo già parlato .

Iniziamo caricando i dati:

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
digits.data.shape

Ricordiamo che i dati sono costituiti da immagini di 8×8 pixel, il che significa che sono a 64 dimensioni. Per ottenere un'intuizione sulle relazioni tra questi punti, possiamo utilizzare PCA per proiettarli su un numero di dimensioni più gestibile, diciamo due:

In [ ]:
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(digits.data)
print(digits.data.shape)
print(projected.shape)

Ora possiamo tracciare le prime due componenti principali di ciascun punto per conoscere i dati:

In [ ]:
plt.scatter(projected[:, 0], projected[:, 1],
            c=digits.target, edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('rainbow', 10))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar()

Ricordiamo cosa significano questi componenti: i dati completi sono una nuvola di punti a 64 dimensioni e questi punti sono la proiezione di ciascun punto dati lungo le direzioni con la varianza maggiore. In sostanza, abbiamo trovato l'allungamento e la rotazione ottimali nello spazio a 64 dimensioni che ci consentono di vedere la disposizione delle cifre in due dimensioni, e lo abbiamo fatto in modo non supervisionato, cioè senza riferimento alle etichette.

### Cosa significano i componenti? 

Possiamo andare un po' oltre e cominciare a chiederci cosa significano le dimensioni ridotte . Questo significato può essere compreso in termini di combinazioni di vettori di base. A esempio, ogni immagine nel set di training è definita da una raccolta di valori di 64 pixel, che chiameremo vettore X:

$$
x = [x_1, x_2, x_3 \cdots x_{64}]
$$

Un modo in cui possiamo gestirli è in termini di pixel, cioè, per costruire l'immagine, moltiplichiamo ciascun elemento del vettore per il pixel che descrive, quindi sommiamo i risultati per costruire l'immagine:

$$
{\rm image}(x) = x_1 \cdot{\rm (pixel~1)} + x_2 \cdot{\rm (pixel~2)} + x_3 \cdot{\rm (pixel~3)} \cdots x_{64} \cdot{\rm (pixel~64)}
$$

Un modo in cui potremmo immaginare di ridurre la dimensione di questi dati è quello di azzerare tutti questi vettori di base tranne alcuni. A esempio, se utilizziamo solo i primi otto pixel, otteniamo una proiezione dei dati in otto dimensioni, ma non riflette molto l'intera immagine: abbiamo eliminato quasi il 90% dei pixel!

![](dati/img/digits-pixel-components.png)


La riga superiore di pannelli mostra i singoli pixel, mentre la riga inferiore mostra il contributo cumulativo di questi pixel alla costruzione dell'immagine. Utilizzando solo otto componenti basati sui pixel, possiamo costruire solo una piccola porzione dell'immagine da 64 pixel. Se continuassimo questa sequenza e utilizzassimo tutti i 64 pixel, recupereremo l'immagine originale.

Ma la rappresentazione in pixel non è l’unica scelta di base, possiamo infatti anche utilizzare altre funzioni di base, che contengono ciascuna un contributo predefinito da ciascun pixel, e scrivere qualcosa di simile

$$
image(x) = {\rm mean} + x_1 \cdot{\rm (basis~1)} + x_2 \cdot{\rm (basis~2)} + x_3 \cdot{\rm (basis~3)} \cdots
$$

La PCA può essere pensata come un processo di scelta delle funzioni di base ottimali, in modo tale che sommare solo le prime di esse è sufficiente per ricostruire adeguatamente la maggior parte degli elementi nel set di dati. Le componenti principali, che fungono da rappresentazione a bassa dimensionalità dei nostri dati, sono semplicemente i coefficienti che moltiplicano ciascuno degli elementi di questa serie. Questa figura mostra una rappresentazione simile della ricostruzione di questa cifra utilizzando la media più le prime otto funzioni base PCA:

![](dati/img/digits-pca-components.png)


A differenza della base pixel, la base PCA permette di recuperare le caratteristiche salienti dell'immagine in ingresso con solo una media più otto componenti! La quantità di ciascun pixel in ciascun componente è il corollario dell'orientamento del vettore nel nostro esempio bidimensionale. Questo è il senso in cui la PCA fornisce una rappresentazione a bassa dimensionalità dei dati: scopre un insieme di funzioni di base che sono più efficienti della base pixel nativa dei dati di input.

### Scelta del numero di componenti

Una parte vitale dell’utilizzo pratico della PCA è la capacità di stimare quanti componenti sono necessari per descrivere i dati. Ciò può essere determinato osservando il rapporto di varianza spiegata cumulativa in funzione del numero di componenti:

In [ ]:
pca = PCA().fit(digits.data)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')

Come già visto, questa curva quantifica quanta parte della varianza totale a 64 dimensioni è contenuta nella prima N componenti. A esempio, vediamo che con le cifre i primi 10 componenti contengono circa il 75% della varianza, mentre sono necessari circa 50 componenti per descrivere quasi il 100% della varianza.

Qui vediamo che la nostra proiezione bidimensionale perde molte informazioni (misurate dalla varianza spiegata) e che avremmo bisogno di circa 20 componenti per conservare il 90% della varianza. Osservare questo grafico per un set di dati ad alta dimensione può aiutarci a comprendere il livello di ridondanza presente in più osservazioni.

## PCA come filtraggio del rumore

La PCA può essere utilizzata anche come approccio di filtraggio per dati rumorosi. L'idea è questa: qualsiasi componente con una varianza molto maggiore dell'effetto del rumore dovrebbe essere relativamente non influenzato dal rumore. Quindi, se ricostruiamo i dati utilizzando solo il sottoinsieme più grande di componenti principali, dovremmo mantenere preferenzialmente il segnale ed eliminare il rumore.

Vediamo come appare con i dati in cifre. Per prima cosa tracciamo diversi dati di input privi di rumore:

In [ ]:
def plot_digits(data):
    fig, axes = plt.subplots(4, 10, figsize=(10, 4),
                             subplot_kw={'xticks':[], 'yticks':[]},
                             gridspec_kw=dict(hspace=0.1, wspace=0.1))
    for i, ax in enumerate(axes.flat):
        ax.imshow(data[i].reshape(8, 8),
                  cmap='binary', interpolation='nearest',
                  clim=(0, 16))
plot_digits(digits.data)

Ora aggiungiamo del rumore casuale per creare un set di dati rumoroso e ritracciarlo:

In [ ]:
rng = np.random.default_rng(42)
rng.normal(10, 2)

In [ ]:
rng = np.random.default_rng(42)
noisy = rng.normal(digits.data, 4)
plot_digits(noisy)

È chiaro a prima vista che le immagini sono rumorose e contengono pixel spuri. Addestriamo una PCA sui dati rumorosi, richiedendo che la proiezione preservi il 50% della varianza:

In [ ]:
pca = PCA(0.50).fit(noisy)
pca.n_components_

Qui il 50% della varianza ammonta a 12 componenti principali. Ora calcoliamo questi componenti e quindi utilizziamo l'inverso della trasformazione per ricostruire le cifre filtrate

In [ ]:
components = pca.transform(noisy)
filtered = pca.inverse_transform(components)
plot_digits(filtered)

Questa proprietà di preservazione del segnale/filtro del rumore rende la PCA una routine di selezione delle funzionalità molto utile: ad esempio, invece di addestrare un classificatore su dati a dimensione molto elevata, potremmo invece addestrare il classificatore sulla rappresentazione a dimensione inferiore, che servirà automaticamente a filtrare rumore casuale negli ingressi.

## Esempio

Nella spiegazione delle macchine vettoriali di supporto, abbiamo esplorato un esempio di utilizzo di una proiezione PCA come selettore di funzionalità per il riconoscimento facciale, ora daremo uno sguardo indietro ed esploreremo un po’ di più ciò che è successo. Ricordiamo che stavamo utilizzando il set di dati Labeled Faces in the Wild reso disponibile tramite Scikit-Learn:

In [ ]:
from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people(min_faces_per_person=60)
print(faces.target_names)
print(faces.images.shape)

Diamo un'occhiata agli assi principali che abbracciano questo set di dati. Poiché si tratta di un set di dati di grandi dimensioni, utilizzeremo Randomized PCA che contiene un metodo randomizzato per approssimare il primo N di componenti principali molto più rapidamente dello PCA stimatore standard, e quindi è molto utile per dati ad alta dimensionalità (qui, una dimensionalità di quasi 3.000). Daremo quindi uno sguardo ai primi 150 componenti:

In [ ]:
pca = PCA(150, svd_solver='randomized', random_state=42)
pca.fit(faces.data)

In questo caso, può essere interessante visualizzare le immagini associate ai primi diversi componenti principali (questi componenti sono tecnicamente noti come "autovettori", quindi questi tipi di immagini sono spesso chiamati "eigenfaces"). Come possiamo vedere in questa figura, i risultati sono inquietanti:

In [ ]:
fig, axes = plt.subplots(3, 8, figsize=(9, 4),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i, ax in enumerate(axes.flat):
    ax.imshow(pca.components_[i].reshape(62, 47), cmap='bone')

I risultati sono molto interessanti e ci danno un'idea di come variano le immagini: a esempio, i primi eigenfaces (dall'alto a sinistra) sembrano essere associati all'angolo di illuminazione sul viso, e successivamente i vettori principali sembrano selezionare alcune caratteristiche, come occhi, naso e labbra. Diamo un'occhiata alla varianza cumulativa di questi componenti per vedere quanta informazione sui dati viene preservata dalla proiezione:

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')

Vediamo che questi 150 componenti rappresentano poco più del 90% della varianza. Ciò ci porterebbe a credere che utilizzando questi 150 componenti recupereremo la maggior parte delle caratteristiche essenziali dei dati. Per renderlo più concreto, possiamo confrontare le immagini di input con le immagini ricostruite da questi 150 componenti:

In [ ]:
# Compute the components and projected faces
pca = pca.fit(faces.data)
components = pca.transform(faces.data)
projected = pca.inverse_transform(components)

In [ ]:
# Plot the results
fig, ax = plt.subplots(2, 10, figsize=(10, 2.5),
                       subplot_kw={'xticks':[], 'yticks':[]},
                       gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i in range(10):
    ax[0, i].imshow(faces.data[i].reshape(62, 47), cmap='binary_r')
    ax[1, i].imshow(projected[i].reshape(62, 47), cmap='binary_r')

ax[0, 0].set_ylabel('full-dim\ninput')
ax[1, 0].set_ylabel('150-dim\nreconstruction')

La riga superiore qui mostra le immagini di input, mentre la riga inferiore mostra la ricostruzione delle immagini da appena 150 delle circa 3.000 caratteristiche iniziali. Questa visualizzazione chiarisce il motivo per cui la selezione delle funzionalità PCA utilizzata nella spiegazione delle Support Vector Machines ha avuto così tanto successo: sebbene riduca la dimensionalità dei dati di quasi un fattore 20, le immagini proiettate contengono informazioni sufficienti che potremmo, a occhio, riconoscere gli individui nell'immagine. Ciò significa che il nostro algoritmo di classificazione deve essere addestrato su dati a 150 dimensioni anziché su dati a 3.000 dimensioni, il che, a seconda del particolare algoritmo che scegliamo, può portare a una classificazione molto più efficiente.

## Conclusione su PCA

In questa sezione abbiamo discusso l'uso dell'analisi delle componenti principali per la riduzione della dimensionalità, per la visualizzazione di dati ad alta dimensionalità, per il filtraggio del rumore e per la selezione delle caratteristiche all'interno di dati ad alta dimensionalità. Grazie alla versatilità e all’interpretabilità della PCA, è stato dimostrato che è efficace in un’ampia varietà di contesti e discipline. Dato qualsiasi set di dati ad alta dimensione, molto spesso conviene utilizzare PCA per visualizzare la relazione tra i punti (come abbiamo fatto con le cifre), per comprendere la varianza principale nei dati (come abbiamo fatto con le eigenfaces) e per comprendere la dimensionalità intrinseca (tracciando il rapporto di varianza spiegato). Certamente la PCA non è utile per ogni set di dati ad alta dimensionalità, ma offre un percorso semplice ed efficiente per ottenere informazioni dettagliate su dati ad alta dimensionalità.

### Il principale punto debole della PCA 
è che tende a essere fortemente influenzato da valori anomali nei dati. Per questo motivo sono state sviluppate molte varianti robuste della PCA, molte delle quali agiscono per scartare in modo iterativo i punti dati che sono scarsamente descritti dai componenti iniziali. Scikit-Learn contiene un paio di varianti interessanti su PCA, inclusi RandomizedPCA e Sparse PCA, entrambi anche nel sottomodulo sklearn.decomposition . Randomized PCA, che abbiamo visto in precedenza, utilizza un metodo non deterministico per approssimare rapidamente i primi componenti principali in dati ad altissima dimensionalità, SparsePCA invece introduce al contempo un termine di regolarizzazione che serve a imporre la scarsità dei componenti.

# Esercizio 1

Utilizzate l'algoritmo k-Nearest Neighbors (k-NN) e PCA (Principal Component Analysis) per la riduzione delle dimensioni sul dataset "Breast Cancer Wisconsin (Diagnostic)" disponibile nella libreria scikit-learn.

### Passaggi
- Caricamento del dataset:

1. Caricare il dataset "Breast Cancer Wisconsin (Diagnostic)" dalla libreria scikit-learn.
2. Esplorare il dataset per comprendere le caratteristiche presenti, i loro tipi e la distribuzione delle classi di output.

- Preprocessing dei dati:

1. Dividere il dataset in features (variabili indipendenti) e target (variabile dipendente).
2. Dividere il dataset in training set e test set utilizzando una proporzione del 70-30.

- Standardizzazione dei dati:

1. Standardizzare le features utilizzando lo StandardScaler di scikit-learn.

- PCA (Principal Component Analysis):

1. Utilizzare la PCA per ridurre la dimensionalità del dataset mantenendo il 95% della varianza totale.

- Creazione del modello k-NN:

1. Creare un modello k-NN specificando il numero di vicini desiderato.

- Addestramento del modello:

1. Addestrare il modello k-NN sul training set.

- Valutazione del modello:

1. Valutare le prestazioni del modello utilizzando il test set.
2. Calcolare l'accuratezza del modello.
3. Visualizzare il report di classificazione che include precision, recall e F1-score per ogni classe.
4. Visualizzare la matrice di confusione per valutare le prestazioni del modello.

# Esercizio 2

Utilizzate l'algoritmo k-Nearest Neighbors (k-NN) e PCA (Principal Component Analysis) per la riduzione delle dimensioni sul dataset "Fashion MNIST" disponibile nella libreria scikit-learn.

### Passaggi
- Caricamento del dataset:

1. Caricare il dataset "Fashion MNIST" dalla libreria scikit-learn.
2. Esplorare il dataset per comprendere le caratteristiche presenti, i loro tipi e la distribuzione delle classi di output.

- Preprocessing dei dati:

1. Dividere il dataset in features (immagini) e target (etichette dei capi di abbigliamento).
2. Dividere il dataset in training set e test set utilizzando una proporzione del 80-20.

- Creazione del modello k-NN:

1. Creare un modello k-NN specificando il numero di vicini desiderato.

- Addestramento del modello:

1. Addestrare il modello k-NN sul training set.

- Valutazione del modello:

1. Valutare le prestazioni del modello utilizzando il test set.
2. Calcolare l'accuratezza del modello.
3. Visualizzare il report di classificazione che include precision, recall e F1-score per ogni classe.
4. Visualizzare la matrice di confusione per valutare le prestazioni del modello.

